**Mount to Google Drive - Uploaded Github Repo**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install dependencies

In [2]:
!pip install --quiet pandas numpy matplotlib seaborn torch torchvision facenet-pytorch insightface pytorch-metric-learning xgboost scikit-learn opencv-python onnxruntime mozuma


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB

In [3]:
import os
import ssl
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

from xgboost import XGBRegressor
from facenet_pytorch import InceptionResnetV1, MTCNN

import torch

# Ignore SSL certificate verification
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [4]:
 ls -l /content

total 8
drwx------ 6 root root 4096 May 28 22:54 drive/
drwxr-xr-x 1 root root 4096 May 23 13:39 sample_data/


In [5]:
#!unzip /content/drive/MyDrive/Final_Project_Repo.zip -d /content/drive/MyDrive/
%cd /content/drive/MyDrive/Final_Project_Repo

/content/drive/MyDrive/Final_Project_Repo


In [6]:
ls -lh

total 6.7M
-rw------- 1 root root 3.4K May 15 20:13 app.py
-rw------- 1 root root 8.8K May 15 20:13 bmi_multimodal_training.py
drwx------ 2 root root 4.0K May 25 21:22 Data/
-rw------- 1 root root 5.4M May 27 20:54 embeddings_train.npz
-rw------- 1 root root 2.2K May 15 20:13 face_detection.py
-rw------- 1 root root 968K May 25 22:43 Feature_Add.csv
-rw------- 1 root root  11K May 15 20:13 fine_tune_model.py
drwx------ 2 root root 4.0K May 25 21:22 __pycache__/
-rw------- 1 root root 1.1K May 15 20:13 README.md
-rw------- 1 root root 3.1K May 15 20:13 requirements.txt
-rw------- 1 root root  233 May 15 20:13 run_bmi_detector_app.sh
drwx------ 2 root root 4.0K May 25 21:22 runs/
drwx------ 2 root root 4.0K May 25 21:22 saved_models/
-rw------- 1 root root 273K May 15 20:13 tensor_model_training.ipynb
drwx------ 2 root root 4.0K May 25 21:22 venv/
drwx------ 2 root root 4.0K May 25 21:22 wandb/
-rw------- 1 root root  95K May 19 00:47 xgboost_pipeline.ipynb


In [78]:
# Paths
IMAGE_DIR = "Data/Images"
existing_image_files = os.listdir(IMAGE_DIR)

# Load CSV
df = pd.read_csv("Data/data.csv").drop(columns=["Unnamed: 0"])
print(f"Loaded {df.shape[0]} rows from CSV")

# Filter out rows with missing image files
df = df[df["name"].isin(existing_image_files)]
print(f"Filtered to {df.shape[0]} rows with existing images")

# Split into training and testing sets
training_data_df = df[df['is_training'] == 1]
testing_data_df = df[df['is_training'] == 0]
print(f"Training data shape: {training_data_df.shape}")
print(f"Testing data shape: {testing_data_df.shape}")
df.head(5)

Loaded 4206 rows from CSV
Filtered to 3962 rows with existing images
Training data shape: (3210, 4)
Testing data shape: (752, 4)


,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
6,25.845588,Female,1,img_6.bmp


In [8]:
# Model setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(image_size=160, margin=20, device=device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [9]:
def process_row(row):
    """
    Helper function to process each row of the DataFrame using with multi-threading.
    """
    img_path = os.path.join(IMAGE_DIR, row['name'])
    try:
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is not None:
            face = face.unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = resnet(face).cpu().numpy().flatten()
            return embedding, row['bmi'], row['name']
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
    return None

def multi_threaded_embedding_extraction(df):
    """
    Embedding extraction for each image with multi-threading.
    """
    # Initialize lists for embeddings and labels
    X = []
    y = []
    filenames = []

    # Use ThreadPoolExecutor to process images in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_row, row) for _, row in df.iterrows()]

        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            if result:
                embedding, bmi, file_name = result
                X.append(embedding)
                y.append(bmi)
                filenames.append(file_name)
    return np.array(X), np.array(y), filenames

# Extract embeddings for training data
training_embeddings, training_labels, training_filenames = multi_threaded_embedding_extraction(training_data_df)

100%|██████████| 3210/3210 [04:54<00:00, 10.89it/s]


In [10]:
testing_embeddings, testing_labels, testing_filenames = multi_threaded_embedding_extraction(testing_data_df)

100%|██████████| 752/752 [01:16<00:00,  9.80it/s]


In [11]:
# Save embeddings as numpy arrays
X = np.array(training_embeddings)
y = np.array(training_labels)

# Split & train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
ls -lh saved_models/

total 84M
-rw------- 1 root root  37M May 15 20:13 arcface_bmi_embedder.pth
-rw------- 1 root root 690K May 28 21:36 best_bmi_model_xgboost.joblib
-rw------- 1 root root 716K May 15 20:13 best_bmi_model_xgboost.pkl
-rw------- 1 root root  43M May 15 20:13 best_bmi_multimodal_model.pth
-rw------- 1 root root 2.2M May 25 23:40 mlp_bmi_model_20250525_234059.joblib


**RECREATE PREVIOUS BEST MODEL**

In [ ]:
from joblib import load
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Load saved model
model_path = "saved_models/best_bmi_model_xgboost.joblib"
best_bmi_model = load(model_path)

# Prepare test data
X_test = np.array(testing_embeddings)
y_test = np.array(testing_labels)

# Make predictions
y_pred = best_bmi_model.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
#rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# Output
print(f"Loaded model from: {model_path}")
print(f"MAE: {mae:.2f}")
#print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:30:46] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


ValueError: Feature shape mismatch, expected: 535, got 512

**Get Combination - Train Extracted Features from Fairface and Facial Landmark Dlib**



*   Embeddings from VggFace2 on Data
*   Race / Gender / Age Prediction from FairFace
*   Facial Landmark features extracted previously from dlib



In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [13]:
%cd /content/drive/MyDrive/Final_Project_Repo

/content/drive/MyDrive/Final_Project_Repo


In [14]:
!ls -lh /content/drive/MyDrive/Final_Project_Repo

total 6.7M
-rw------- 1 root root 3.4K May 15 20:13 app.py
-rw------- 1 root root 8.8K May 15 20:13 bmi_multimodal_training.py
drwx------ 3 root root 4.0K May 25 21:22 Data
-rw------- 1 root root 5.4M May 27 20:54 embeddings_train.npz
-rw------- 1 root root 2.2K May 15 20:13 face_detection.py
-rw------- 1 root root 968K May 25 22:43 Feature_Add.csv
-rw------- 1 root root  11K May 15 20:13 fine_tune_model.py
drwx------ 2 root root 4.0K May 25 21:22 __pycache__
-rw------- 1 root root 1.1K May 15 20:13 README.md
-rw------- 1 root root 3.1K May 15 20:13 requirements.txt
-rw------- 1 root root  233 May 15 20:13 run_bmi_detector_app.sh
drwx------ 2 root root 4.0K May 25 21:22 runs
drwx------ 2 root root 4.0K May 25 21:22 saved_models
-rw------- 1 root root 273K May 15 20:13 tensor_model_training.ipynb
drwx------ 2 root root 4.0K May 25 21:22 venv
drwx------ 2 root root 4.0K May 25 21:22 wandb
-rw------- 1 root root  95K May 19 00:47 xgboost_pipeline.ipynb


In [ ]:
### Upload Manually ###
from google.colab import files
uploaded = files.upload()

Saving Feature_Add.csv to Feature_Add.csv


In [15]:
import pandas as pd
import io
added = pd.read_csv("Feature_Add.csv")

In [16]:
added

,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,image,race,pred_gender,age,key,bmi,gender,is_training,name
0,237.002110,210.287898,237.008439,52.009614,120.016666,58.034473,62.008064,46.173586,42.107007,1.127036,2.307586,img_1066_face0.jpg,White,Female,20-29,img_1066,41.191406,Female,1,img_1066.bmp
1,226.035395,200.010000,220.002273,51.000000,104.076895,38.052595,62.072538,43.011626,41.048752,1.130120,2.040723,img_1585_face0.jpg,White,Male,20-29,img_1585,24.658895,Male,1,img_1585.bmp
2,241.101638,209.193690,239.075302,42.011903,87.005747,32.062439,65.000000,42.107007,43.011626,1.152528,2.070978,img_3852_face0.jpg,White,Female,20-29,img_3852,39.151259,Female,0,img_3852.bmp
3,229.490741,198.494332,229.176788,53.037722,80.056230,12.000000,54.083269,42.011903,41.012193,1.156158,1.509421,img_1857_face0.jpg,East Asian,Male,30-39,img_1857,25.845588,Male,1,img_1857.bmp
4,235.552542,201.613492,235.766834,48.041649,77.524190,27.892651,60.299254,34.014703,41.194660,1.168337,1.613687,img_4196_face0.jpg,White,Male,20-29,img_4196,36.243556,Male,0,img_4196.bmp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,264.047344,194.041233,260.069222,43.000000,77.025970,22.000000,60.008333,49.010203,45.099889,1.360780,1.791302,img_70_face0.jpg,White,Male,20-29,img_70,37.973684,Male,1,img_70.bmp
3708,255.096060,207.060378,251.017928,50.039984,113.110565,40.112342,63.031738,41.303753,38.000000,1.231989,2.260404,img_4118_face0.jpg,White,Female,20-29,img_4118,34.747659,Female,0,img_4118.bmp
3709,220.445004,153.834326,216.187419,50.039984,89.140339,16.124515,61.032778,41.773197,40.311289,1.433003,1.781382,img_1810_face0.jpg,White,Female,20-29,img_1810,34.110059,Female,1,img_1810.bmp
3710,244.000000,194.023194,242.008264,56.008928,113.017698,25.000000,66.000000,46.010868,42.000000,1.257582,2.017851,img_3815_face0.jpg,White,Female,20-29,img_3815,26.628788,Female,0,img_3815.bmp


In [79]:
# Paths
import os
IMAGE_DIR = "Data/Images"
existing_image_files = os.listdir(IMAGE_DIR)

# Load CSV
df = pd.read_csv("Data/data.csv").drop(columns=["Unnamed: 0"])
print(f"Loaded {df.shape[0]} rows from CSV")

# Filter out rows with missing image files
df = df[df["name"].isin(existing_image_files)]
print(f"Filtered to {df.shape[0]} rows with existing images")

Loaded 4206 rows from CSV
Filtered to 3962 rows with existing images


In [80]:
### Join with Data given ###
df2 = pd.merge(df, added, on=['gender', 'bmi', 'name', 'is_training'], how='inner')

In [81]:
df2.head()

,bmi,gender,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,image,race,pred_gender,age,key
0,34.207396,Male,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,img_0_face0.jpg,White,Male,30-39,img_0
1,26.453720,Male,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,img_1_face0.jpg,White,Male,20-29,img_1
2,34.967561,Female,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,img_2_face0.jpg,White,Female,20-29,img_2
3,22.044766,Female,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,img_3_face0.jpg,White,Female,20-29,img_3
4,25.845588,Female,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,img_6_face0.jpg,White,Female,10-19,img_6


In [82]:
df2.drop(columns=['key', 'image'], inplace=True)

In [83]:
df2.head()

,bmi,gender,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,race,pred_gender,age
0,34.207396,Male,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,White,Male,30-39
1,26.453720,Male,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,White,Male,20-29
2,34.967561,Female,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,White,Female,20-29
3,22.044766,Female,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,White,Female,20-29
4,25.845588,Female,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,White,Female,10-19


In [22]:
#df2.drop(columns="gender", inplace=True)

In [84]:
df2.head()

,bmi,gender,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,right_eye_width,face_width_to_height,mouth_to_nose_ratio,race,pred_gender,age
0,34.207396,Male,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,43.046487,1.374692,1.940412,White,Male,30-39
1,26.453720,Male,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,43.185646,1.227341,1.832808,White,Male,20-29
2,34.967561,Female,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,44.045431,1.240158,2.146238,White,Female,20-29
3,22.044766,Female,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,41.012193,1.128941,2.316792,White,Female,20-29
4,25.845588,Female,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,47.010637,1.218169,1.937610,White,Female,10-19


In [85]:
### One Hot Encode ###
race_dummies = pd.get_dummies(df2['race'], prefix='race').astype(int)
gender_dummies = pd.get_dummies(df2['pred_gender'], prefix='gender').astype(int)
age_dummies = pd.get_dummies(df2['age'], prefix='age').astype(int)

# Combine with original numeric features
final_dat_encoded = pd.concat([
    df2.drop(['race', 'pred_gender', 'age'], axis=1),
    race_dummies,
    gender_dummies,
    age_dummies
], axis=1)

final_dat_encoded

,bmi,gender,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,...,race_Latino_Hispanic,race_White,gender_Female,gender_Male,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59
0,34.207396,Male,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,...,0,1,0,1,0,0,0,1,0,0
1,26.453720,Male,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,...,0,1,0,1,0,1,0,0,0,0
2,34.967561,Female,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,...,0,1,1,0,0,1,0,0,0,0
3,22.044766,Female,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,...,0,1,1,0,0,1,0,0,0,0
4,25.845588,Female,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,...,0,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,34.078947,Male,0,img_4201.bmp,254.017716,200.010000,250.017999,48.010416,108.018517,37.000000,...,0,1,0,1,0,1,0,0,0,0
3708,34.564776,Female,0,img_4202.bmp,239.033470,210.608642,240.002083,44.102154,98.508883,32.062439,...,0,1,1,0,0,1,0,0,0,0
3709,27.432362,Female,0,img_4203.bmp,241.101638,208.021633,236.002119,41.012193,102.004902,52.038447,...,1,0,1,0,0,1,0,0,0,0
3710,40.492800,Male,0,img_4204.bmp,232.053873,199.062804,231.002164,51.009803,88.141931,17.262677,...,0,1,0,1,0,0,0,1,0,0


In [96]:
# Split into training and testing sets
training_data_df = final_dat_encoded[final_dat_encoded['is_training'] == 1]
testing_data_df = final_dat_encoded[final_dat_encoded['is_training'] == 0]
print(f"Training data shape: {training_data_df.shape}")
print(f"Testing data shape: {testing_data_df.shape}")

Training data shape: (3008, 27)
Testing data shape: (704, 27)


In [ ]:
# Model setup - Extract Embeddings with VGGFACE2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(image_size=160, margin=20, device=device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
def process_row(row):
    """
    Helper function to process each row of the DataFrame using with multi-threading.
    """
    img_path = os.path.join(IMAGE_DIR, row['name'])
    try:
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is not None:
            face = face.unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = resnet(face).cpu().numpy().flatten()
            return embedding, row['bmi'], row['name']
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
    return None

def multi_threaded_embedding_extraction(df):
    """
    Embedding extraction for each image with multi-threading.
    """
    # Initialize lists for embeddings and labels
    X = []
    y = []
    filenames = []

    # Use ThreadPoolExecutor to process images in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_row, row) for _, row in df.iterrows()]

        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            if result:
                embedding, bmi, file_name = result
                X.append(embedding)
                y.append(bmi)
                filenames.append(file_name)
    return np.array(X), np.array(y), filenames

# Extract embeddings for training data
training_embeddings, training_labels, training_filenames = multi_threaded_embedding_extraction(training_data_df)

100%|██████████| 3008/3008 [06:18<00:00,  7.94it/s]


In [87]:
embedding_df = pd.DataFrame(training_embeddings)
embedding_df['name'] = training_filenames

In [88]:
embedding_df

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,name
0,-0.018495,0.025302,-0.002864,-0.060063,0.002269,0.031754,-0.010016,-0.067670,0.112952,0.000151,...,0.000151,0.085531,0.017921,0.020728,-0.083109,-0.008004,0.000996,-0.044594,-0.017960,img_8.bmp
1,-0.092354,-0.016411,-0.021643,-0.083864,0.050211,-0.003199,0.048190,-0.061914,0.126567,0.020999,...,-0.004851,-0.012994,-0.001783,-0.020546,-0.025670,-0.021894,0.007396,0.036082,0.021365,img_7.bmp
2,-0.032583,0.005906,0.048234,0.041946,-0.013243,0.052662,-0.087507,0.046184,0.063188,-0.037088,...,0.026378,-0.040784,0.011327,0.045461,-0.110210,0.033463,0.015070,-0.030911,0.028445,img_2.bmp
3,-0.064325,-0.031137,-0.031269,-0.033018,0.035816,-0.030125,-0.012394,-0.048786,0.042607,0.022765,...,0.021143,-0.082897,-0.001238,-0.002314,-0.019928,-0.060588,-0.017504,0.102679,0.035158,img_6.bmp
4,0.021421,-0.065005,0.036096,0.067527,0.026900,0.064042,-0.082973,-0.000360,0.013251,-0.023872,...,0.021704,-0.036057,0.042702,0.049972,-0.107530,0.021169,0.007321,-0.040787,-0.034802,img_3.bmp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0.025214,-0.018572,-0.059827,-0.027831,0.012538,-0.072807,-0.043675,0.049532,-0.015904,-0.005515,...,-0.004118,-0.029492,-0.004425,0.008595,0.015558,-0.040983,-0.066267,0.099026,-0.007834,img_3360.bmp
3200,-0.061248,0.005550,-0.026836,0.083047,-0.019360,-0.073779,-0.034171,0.012527,0.023004,-0.079035,...,0.043359,0.008605,-0.011273,0.066960,-0.069991,-0.017267,-0.015751,0.025749,-0.032823,img_3363.bmp
3201,0.051428,0.094767,0.014756,0.001317,0.042908,0.028780,0.000874,0.036979,0.015501,0.008421,...,-0.055746,0.023276,-0.049996,0.035154,0.009024,0.022903,-0.024533,0.008647,0.026862,img_3364.bmp
3202,-0.039881,0.017540,0.008939,-0.025688,-0.001632,0.020124,-0.050197,-0.041656,0.095125,0.030197,...,-0.074009,0.041139,-0.070548,-0.062801,-0.028364,0.085487,0.019831,0.052999,0.076221,img_3356.bmp


In [89]:
### Combine Embeddings and Extracted Features ###
feature_df = final_dat_encoded.drop(columns=['bmi', 'is_training', 'gender'])
merged = pd.merge(embedding_df, feature_df, on='name')
merged = merged.drop_duplicates(subset='name')

In [90]:
merged.head()

,0,1,2,3,4,5,6,7,8,9,...,race_Latino_Hispanic,race_White,gender_Female,gender_Male,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59
0,-0.018495,0.025302,-0.002864,-0.060063,0.002269,0.031754,-0.010016,-0.067670,0.112952,0.000151,...,0,1,1,0,0,1,0,0,0,0
1,-0.092354,-0.016411,-0.021643,-0.083864,0.050211,-0.003199,0.048190,-0.061914,0.126567,0.020999,...,0,1,1,0,0,1,0,0,0,0
2,-0.032583,0.005906,0.048234,0.041946,-0.013243,0.052662,-0.087507,0.046184,0.063188,-0.037088,...,0,1,1,0,0,1,0,0,0,0
3,-0.064325,-0.031137,-0.031269,-0.033018,0.035816,-0.030125,-0.012394,-0.048786,0.042607,0.022765,...,0,1,1,0,1,0,0,0,0,0
4,0.021421,-0.065005,0.036096,0.067527,0.026900,0.064042,-0.082973,-0.000360,0.013251,-0.023872,...,0,1,1,0,0,1,0,0,0,0


In [30]:
### Need These in Real - Time
print(list(merged.columns[511:]))

[511, 'name', 'jaw_width', 'face_height', 'cheekbone_width', 'nose_width', 'mouth_width', 'mouth_height', 'eye_distance', 'left_eye_width', 'right_eye_width', 'face_width_to_height', 'mouth_to_nose_ratio', 'age_10-19', 'age_20-29', 'age_3-9', 'age_30-39', 'age_40-49', 'age_50-59']


In [93]:
merged_full = pd.merge(merged, final_dat_encoded[['name', 'bmi']], on='name', how='inner')
X = merged_full.drop(columns=['name', 'bmi']).values
y = merged_full['bmi'].values

In [95]:
merged_full.head()

,0,1,2,3,4,5,6,7,8,9,...,race_White,gender_Female,gender_Male,age_10-19,age_20-29,age_3-9,age_30-39,age_40-49,age_50-59,bmi
0,-0.018495,0.025302,-0.002864,-0.060063,0.002269,0.031754,-0.010016,-0.067670,0.112952,0.000151,...,1,1,0,0,1,0,0,0,0,30.615858
1,-0.092354,-0.016411,-0.021643,-0.083864,0.050211,-0.003199,0.048190,-0.061914,0.126567,0.020999,...,1,1,0,0,1,0,0,0,0,21.740701
2,-0.032583,0.005906,0.048234,0.041946,-0.013243,0.052662,-0.087507,0.046184,0.063188,-0.037088,...,1,1,0,0,1,0,0,0,0,34.967561
3,-0.064325,-0.031137,-0.031269,-0.033018,0.035816,-0.030125,-0.012394,-0.048786,0.042607,0.022765,...,1,1,0,1,0,0,0,0,0,25.845588
4,0.021421,-0.065005,0.036096,0.067527,0.026900,0.064042,-0.082973,-0.000360,0.013251,-0.023872,...,1,1,0,0,1,0,0,0,0,22.044766


In [71]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [74]:
### Train XGBOOST with GPU ###
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

def train_bmi_regressor_with_cv(X, y, cv_folds=5, n_iter=20, random_state=42):
    """
    Trains an XGBoost regressor using RandomizedSearchCV.
    """
    xgb = XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    random_state=random_state
)

    param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_lambda': [0.1, 1, 10],
        'reg_alpha': [0, 0.1, 1]
    }
    xgb = XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    random_state=42
    )

    scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    search = RandomizedSearchCV(
        xgb,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=scorer,
        cv=cv_folds,
        verbose=3,
        random_state=random_state,
        n_jobs=-1
    )

    search.fit(X, y)

    print(f"Best MAE: {-search.best_score_:.4f}")
    print(f"Best Parameters: {search.best_params_}")

    return search.best_estimator_

best_bmi_model = train_bmi_regressor_with_cv(X, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:45:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:45:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Best MAE: 4.4632
Best Parameters: {'subsample': 0.6, 'reg_lambda': 10, 'reg_alpha': 0.1, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 1.0}


In [104]:
# Step 1: Create embedding DataFrame for test
embedding_test_df = pd.DataFrame(testing_embeddings)
embedding_test_df['name'] = testing_filenames

# Step 2: Prepare feature DataFrame (drop bmi + is_training like in training)
feature_test_df = final_dat_encoded[final_dat_encoded['is_training'] == 0]
feature_test_df = feature_test_df.drop(columns=['bmi', 'is_training', 'gender'])

# Step 3: Merge embeddings + features
merged_test = pd.merge(embedding_test_df, feature_test_df, on='name')
merged_test = merged_test.drop_duplicates(subset='name')  # mirror training

# Step 4: Add BMI values
merged_test_full = pd.merge(merged_test, final_dat_encoded[['name', 'bmi']], on='name', how='inner')

# Step 5: Create X_test and y_test
X_test = merged_test_full.drop(columns=['name', 'bmi']).values
y_test = merged_test_full['bmi'].values

In [105]:
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
import joblib

# Predict
y_pred = best_bmi_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
pearson_corr, _ = pearsonr(y_test, y_pred)

# Save model
#joblib.dump(best_bmi_model, "saved_models/best_bmi_model_xgboost.joblib")

# Print results
print(f"Test MAE: {mae:.2f}")
print(f"Test R²: {r2:.2f}")
print(f"Test Pearson Correlation: {pearson_corr:.2f}")

Test MAE: 5.07
Test R²: 0.41
Test Pearson Correlation: 0.65


In [108]:
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
import pandas as pd

# ---- Merge and align ----
merged_test_full = pd.merge(merged_test, final_dat_encoded[['name', 'bmi', 'gender']], on='name', how='inner')
merged_test_full = merged_test_full.reset_index(drop=True)

# ---- Identify columns ----
names_test = merged_test_full['name']
genders_test = merged_test_full['gender']
X_test = merged_test_full.drop(columns=['name', 'bmi', 'gender']).values
y_test = merged_test_full['bmi'].values

# ---- Predict ----
y_pred = best_bmi_model.predict(X_test)

# ---- Results DataFrame ----
results_df = pd.DataFrame({
    'name': names_test,
    'gender': genders_test,
    'y_true': y_test,
    'y_pred': y_pred
})

# ---- Overall metrics ----
overall_mae = mean_absolute_error(results_df['y_true'], results_df['y_pred'])
overall_r2 = r2_score(results_df['y_true'], results_df['y_pred'])
overall_pearson, _ = pearsonr(results_df['y_true'], results_df['y_pred'])

metrics_list = [{
    'gender': 'Overall',
    'Count': len(results_df),
    'MAE': round(overall_mae, 2),
    'R²': round(overall_r2, 2),
    'Pearson': round(overall_pearson, 2)
}]

# ---- Gender-wise metrics ----
for gender, group in results_df.groupby('gender'):
    y_true = group['y_true']
    y_pred = group['y_pred']

    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    pearson_corr, _ = pearsonr(y_true, y_pred)

    metrics_list.append({
        'gender': gender,
        'Count': len(group),
        'MAE': round(mae, 2),
        'R²': round(r2, 2),
        'Pearson': round(pearson_corr, 2)
    })

# ---- Final Table ----
metrics_df = pd.DataFrame(metrics_list)
print(metrics_df)

    gender  Count   MAE    R²  Pearson
0  Overall    702  5.07  0.41     0.65
1   Female    307  5.16  0.42     0.67
2     Male    395  4.99  0.40     0.65


In [100]:
print("X_test shape:", X_test.shape)
print("merged_test shape:", merged_test.shape)

X_test shape: (702, 535)
merged_test shape: (701, 536)


This XGBoost gives us our highest R^2 and Pearson Correlation. 'best_bmi_model_xgboost' will be used in real-time application.

**Attempt Repeat with Random Forrest**

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer

def train_bmi_regressor_with_cv(X, y, cv_folds=5, n_iter=20, random_state=42):
    """
    Trains a Random Forest regressor using RandomizedSearchCV.
    """
    rf = RandomForestRegressor(random_state=random_state)

    param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False]
    }

    scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    search = RandomizedSearchCV(
        rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=scorer,
        cv=cv_folds,
        verbose=3,
        random_state=random_state,
        n_jobs=-1
    )

    search.fit(X, y)

    print(f"Best MAE: {-search.best_score_:.4f}")
    print(f"Best Parameters: {search.best_params_}")

    return search.best_estimator_

# Example call
best_bmi_model = train_bmi_regressor_with_cv(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_

Best MAE: 4.6841
Best Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15, 'bootstrap': False}


In [39]:
y_pred = best_bmi_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MAE: {mae:.2f}")
print(f"Test R²: {r2:.2f}")

Test MAE: 5.25
Test R²: 0.37


**Multi Layer Perceptron**

In [40]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime
import joblib

# Define model
mlp = MLPRegressor(max_iter=500, random_state=42)

# Optionally tune it
param_dist = {
    'hidden_layer_sizes': [(128,), (256, 128), (256, 128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'learning_rate_init': [1e-3, 5e-4]
}

search = RandomizedSearchCV(
    mlp,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_mean_absolute_error',
    cv=5,
    n_jobs=-1,
    verbose=3,
    random_state=42
)

search.fit(X_train, y_train)
best_mlp = search.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [41]:
print(best_mlp)

MLPRegressor(activation='tanh', hidden_layer_sizes=(256, 128),
             learning_rate_init=0.0005, max_iter=500, random_state=42)


In [43]:
# Predict
y_pred = best_mlp.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
#rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
pearson_corr, _ = pearsonr(y_test, y_pred)

print(f"MAE: {mae:.2f}")
#print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")
print(f"Test Pearson Correlation: {pearson_corr:.2f}")

# Save model
filename = f"saved_models/mlp_bmi_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.joblib"
joblib.dump(best_mlp, filename)
print(f"Saved model to: {filename}")

MAE: 5.19
R² Score: 0.38
Test Pearson Correlation: 0.63
Saved model to: saved_models/mlp_bmi_model_20250528_233003.joblib


The MLP Regressor yielded slightly better results than the XGBoost, will look to implement live as well.

**K Neighbors Regressor - In Theory Similar Faces could yield similar BMI**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


# Tune k
param_grid = {'n_neighbors': [3, 5, 10, 20], 'weights': ['uniform', 'distance']}
knn = KNeighborsRegressor()

search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

best_knn = search.best_estimator_
y_pred = best_knn.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
#rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Best k: {search.best_params_}")
print(f"MAE: {mae:.2f}, R²: {r2:.2f}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best k: {'n_neighbors': 20, 'weights': 'distance'}
MAE: 6.02, R²: 0.18


Did not perform very well.

**Get Embeddings from Best Multi Model**

*   Instead of VGGFace2, use embeddings from fine tuned multi model.
*   Combine with Feature Extraction again.



In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN

# Paths
CSV_FILE = "Data/data.csv"
IMAGE_DIR = "Data/Images"
MODEL_PATH = "saved_models/best_bmi_multimodal_model.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, mtcnn=None):
        self.df = pd.read_csv(csv_file).drop(columns=["Unnamed: 0"], errors="ignore")
        self.df = self.df[self.df["name"].isin(os.listdir(image_dir))].reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.mtcnn = mtcnn
        self.gender_map = {"Male": 0, "Female": 1}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["name"])
        img = Image.open(img_path).convert("RGB")

        if self.mtcnn:
            face = self.mtcnn(img)
            if face is not None:
                img = transforms.ToPILImage()(face.squeeze(0).cpu())

        if self.transform:
            img = self.transform(img)

        gender = torch.tensor(self.gender_map.get(row["gender"], 0), dtype=torch.long)
        bmi = torch.tensor(row["bmi"], dtype=torch.float32)
        name = row["name"]
        return img, gender, bmi, name


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

mtcnn = MTCNN(image_size=224, margin=0, keep_all=False, device=DEVICE)

dataset = EmbeddingDataset(CSV_FILE, IMAGE_DIR, transform, mtcnn)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
model = BMIModel().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 77.6MB/s]


BMIModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [ ]:
all_embeddings = []
all_genders = []
all_bmis = []
all_names = []

with torch.no_grad():
    for imgs, genders, bmis, names in tqdm(loader):
        imgs = imgs.to(DEVICE)
        features = model.backbone(imgs)  # output shape: [batch, 512]

        all_embeddings.append(features.cpu().numpy())
        all_genders.extend(genders.numpy())
        all_bmis.extend(bmis.numpy())
        all_names.extend(names)

X_embed = np.vstack(all_embeddings)

  0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
df_embed = pd.DataFrame(X_embed, columns=[f"emb_{i}" for i in range(X_embed.shape[1])])
df_embed["name"] = all_names

In [ ]:
df_embed.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,name
0,0.600995,1.367645,0.958343,1.725421,1.373243,1.160275,1.367666,1.619963,1.425280,0.922976,...,1.077119,0.769058,0.982828,1.421242,1.456203,1.329003,1.409490,1.380237,1.474673,img_0.bmp
1,0.541694,1.014046,0.936014,0.979777,0.982789,1.085461,0.942146,1.077627,1.155128,0.893812,...,0.772257,0.691906,0.730243,0.955643,1.168557,1.019338,1.104651,0.961610,0.866305,img_1.bmp
2,0.637231,1.462452,0.986425,1.641200,1.454787,1.150098,1.462985,1.818512,1.510316,0.974788,...,1.164059,0.750106,0.938405,1.452415,1.547865,1.447489,1.457058,1.424043,1.423894,img_2.bmp
3,0.571766,1.003972,0.895183,0.661013,0.871605,1.059731,0.874542,0.760897,1.102327,0.895525,...,0.732119,0.642894,0.629657,0.942043,1.068681,0.959939,1.030902,0.685514,0.702877,img_3.bmp
4,0.685844,1.076732,0.969262,1.017292,0.962810,1.040401,0.932953,1.057037,1.165638,0.925112,...,0.858305,0.746823,0.855155,1.026410,1.142503,1.025039,1.109894,1.069155,1.066098,img_6.bmp


In [ ]:
df_embed

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,name
0,0.600995,1.367645,0.958343,1.725421,1.373243,1.160275,1.367666,1.619963,1.425280,0.922976,...,1.077119,0.769058,0.982828,1.421242,1.456203,1.329003,1.409490,1.380237,1.474673,img_0.bmp
1,0.541694,1.014046,0.936014,0.979777,0.982789,1.085461,0.942146,1.077627,1.155128,0.893812,...,0.772257,0.691906,0.730243,0.955643,1.168557,1.019338,1.104651,0.961610,0.866305,img_1.bmp
2,0.637231,1.462452,0.986425,1.641200,1.454787,1.150098,1.462985,1.818512,1.510316,0.974788,...,1.164059,0.750106,0.938405,1.452415,1.547865,1.447489,1.457058,1.424043,1.423894,img_2.bmp
3,0.571766,1.003972,0.895183,0.661013,0.871605,1.059731,0.874542,0.760897,1.102327,0.895525,...,0.732119,0.642894,0.629657,0.942043,1.068681,0.959939,1.030902,0.685514,0.702877,img_3.bmp
4,0.685844,1.076732,0.969262,1.017292,0.962810,1.040401,0.932953,1.057037,1.165638,0.925112,...,0.858305,0.746823,0.855155,1.026410,1.142503,1.025039,1.109894,1.069155,1.066098,img_6.bmp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,1.971611,1.078369,1.027135,0.963322,1.049006,1.015759,1.066443,1.094574,1.232513,1.321844,...,0.843913,1.767110,0.752487,1.869797,1.221443,1.062262,1.185181,0.572460,1.282825,img_4201.bmp
3958,0.720357,1.384935,0.928062,1.106601,1.265422,1.141376,1.270422,1.290477,1.383474,0.918574,...,1.038389,0.587450,0.876827,1.333517,1.400458,1.277313,1.316269,1.176674,1.078920,img_4202.bmp
3959,1.501746,1.087273,0.938779,0.747627,0.966520,1.045377,0.967612,0.884890,1.110080,1.193440,...,0.819231,1.247235,0.647609,1.350169,1.120568,1.029509,1.095479,0.478370,1.023886,img_4203.bmp
3960,0.640009,1.349349,1.020133,1.420380,1.295772,1.176587,1.268245,1.452848,1.342779,0.930941,...,0.991263,0.754986,0.894353,1.372666,1.402175,1.275610,1.328845,1.325978,1.301241,img_4204.bmp


In [ ]:
### Combine with Best multimodal
total_df = final_dat_encoded.merge(df_embed, on="name")

In [ ]:
total_df

,bmi,is_training,name,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,34.207396,1,img_0.bmp,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,...,1.338998,1.077119,0.769058,0.982828,1.421242,1.456203,1.329003,1.409490,1.380237,1.474673
1,26.453720,1,img_1.bmp,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,...,1.012825,0.772257,0.691906,0.730243,0.955643,1.168557,1.019338,1.104651,0.961610,0.866305
2,34.967561,1,img_2.bmp,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,...,1.477117,1.164059,0.750106,0.938405,1.452415,1.547865,1.447489,1.457058,1.424043,1.423894
3,22.044766,1,img_3.bmp,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,...,0.903959,0.732119,0.642894,0.629657,0.942043,1.068681,0.959939,1.030902,0.685514,0.702877
4,25.845588,1,img_6.bmp,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,...,0.970122,0.858305,0.746823,0.855155,1.026410,1.142503,1.025039,1.109894,1.069155,1.066098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,34.078947,0,img_4201.bmp,254.017716,200.010000,250.017999,48.010416,108.018517,37.000000,67.007462,...,1.077384,0.843913,1.767110,0.752487,1.869797,1.221443,1.062262,1.185181,0.572460,1.282825
3708,34.564776,0,img_4202.bmp,239.033470,210.608642,240.002083,44.102154,98.508883,32.062439,58.008620,...,1.265753,1.038389,0.587450,0.876827,1.333517,1.400458,1.277313,1.316269,1.176674,1.078920
3709,27.432362,0,img_4203.bmp,241.101638,208.021633,236.002119,41.012193,102.004902,52.038447,61.032778,...,0.979745,0.819231,1.247235,0.647609,1.350169,1.120568,1.029509,1.095479,0.478370,1.023886
3710,40.492800,0,img_4204.bmp,232.053873,199.062804,231.002164,51.009803,88.141931,17.262677,64.007812,...,1.286235,0.991263,0.754986,0.894353,1.372666,1.402175,1.275610,1.328845,1.325978,1.301241


In [ ]:
X_train = total_df[total_df["is_training"] == 1].drop(columns=["bmi", "name", "is_training"])
y_train = total_df[total_df["is_training"] == 1]["bmi"]
X_test = total_df[total_df["is_training"] == 0].drop(columns=["bmi", "name", "is_training"])
y_test = total_df[total_df["is_training"] == 0]["bmi"]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

def train_bmi_regressor_with_cv(X, y, cv_folds=5, n_iter=20, random_state=42):
    """
    Trains an XGBoost regressor using RandomizedSearchCV.
    """
    xgb = XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    random_state=random_state
)

    param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_lambda': [0.1, 1, 10],
        'reg_alpha': [0, 0.1, 1]
    }
    xgb = XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    random_state=random_state
    )

    scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    search = RandomizedSearchCV(
        xgb,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=scorer,
        cv=cv_folds,
        verbose=3,
        random_state=random_state,
        n_jobs=-1
    )

    search.fit(X, y)

    print(f"Best MAE: {-search.best_score_:.4f}")
    print(f"Best Parameters: {search.best_params_}")

    return search.best_estimator_

best_bmi_model = train_bmi_regressor_with_cv(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:25:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:25:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Best MAE: 0.9319
Best Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.6}


In [ ]:
y_pred = best_bmi_model.predict(X_test)

# Metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

MAE: 5.51
R² Score: 0.29


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:26:55] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:26:55] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


We see that the model performs very well on the training set, but is overfitted significantly. This can't be used in production.

**NPZ Embeddings VGGFace2 Fine-Tuned**


*   VggFace2 Fine Tuned w/ Data Embeddings
*   Combine with Extracted Features



In [ ]:
from google.colab import files

# Upload file
uploaded = files.upload()

Saving embeddings_train.npz to embeddings_train.npz


In [ ]:
import numpy as np

# Load the .npz file
vggface_2_embeddings = np.load('embeddings_train.npz')

In [ ]:
embeddings = vggface_2_embeddings['embeddings']
image_paths = vggface_2_embeddings['image_paths']

In [ ]:
print("Embeddings shape:", embeddings.shape)
print("Number of image paths:", len(image_paths))
print("Sample pair:", image_paths[0], embeddings[0])

Embeddings shape: (2653, 512)
Number of image paths: 2653
Sample pair: img_853.bmp [ 6.63610268e-03 -3.79855856e-02 -6.29770458e-02  8.32313970e-02
  1.14124110e-02  7.71408901e-02 -3.60384770e-02  2.38057822e-02
 -6.75131381e-02 -2.22825492e-03 -2.43721716e-02  4.61968370e-02
 -5.04148565e-02 -4.19255383e-02  1.55235045e-02 -8.66638031e-03
  4.89607565e-02  4.34301570e-02  7.80694559e-02 -1.00164272e-01
 -5.22275269e-02  3.31340320e-02  7.36494139e-02  2.93335244e-02
  2.43955161e-02  4.15538698e-02  5.02946451e-02 -2.16420256e-02
  4.84542595e-03  2.44402941e-02 -8.49217996e-02 -1.47603126e-02
 -1.38227195e-02 -4.78292117e-03 -9.74576268e-03  4.24901135e-02
 -3.25297527e-02  4.35232790e-03 -6.37062863e-02  7.30154142e-02
  5.36751328e-03  2.10661795e-02 -9.25560854e-03 -3.61254662e-02
  9.98884067e-03  1.42442510e-02  1.51169924e-02  6.12481534e-02
 -7.66614452e-02  4.23080660e-02  1.89307090e-02  7.09522888e-02
  4.85926121e-02 -2.41286121e-03 -7.90472850e-02  3.72676253e-02
  4.841

In [ ]:
df = pd.DataFrame({
    'image_path': image_paths,
    'embedding': list(embeddings)
})

In [ ]:
df

,image_path,embedding
0,img_853.bmp,"[0.0066361027, -0.037985586, -0.062977046, 0.0..."
1,img_3130.bmp,"[0.05224229, -0.06596138, -0.06757551, 0.05203..."
2,img_2453.bmp,"[0.002881161, -0.06419164, -0.09466221, 0.0548..."
3,img_2169.bmp,"[-0.0526027, -0.025005912, -0.08732312, 0.0363..."
4,img_369.bmp,"[-0.07175579, -0.013676618, -0.075680375, 0.05..."
...,...,...
2648,img_387.bmp,"[-0.042541582, 0.031318545, -0.07897792, 0.018..."
2649,img_2260.bmp,"[-0.02929606, -0.018232176, -0.07884138, 0.120..."
2650,img_505.bmp,"[0.019770129, -0.012468107, -0.08492249, 0.074..."
2651,img_2591.bmp,"[-0.017279735, -0.010304326, -0.08675479, 0.07..."


In [ ]:
final_df_embed = final_dat_encoded.merge(df, left_on="name", right_on="image_path").drop(columns = ['name', 'image_path'])

In [ ]:
final_df_embed.columns

Index(['bmi', 'is_training', 'jaw_width', 'face_height', 'cheekbone_width',
       'nose_width', 'mouth_width', 'mouth_height', 'eye_distance',
       'left_eye_width', 'right_eye_width', 'face_width_to_height',
       'mouth_to_nose_ratio', 'race_Black', 'race_East Asian',
       'race_Latino_Hispanic', 'race_White', 'gender_Female', 'gender_Male',
       'age_10-19', 'age_20-29', 'age_3-9', 'age_30-39', 'age_40-49',
       'age_50-59', 'embedding'],
      dtype='object')

In [ ]:
embedding_df = pd.DataFrame(final_df_embed['embedding'].tolist())

# Rename columns for clarity
embedding_df.columns = [f'embedding_{i}' for i in range(embedding_df.shape[1])]

# Combine with the original df (drop old embedding column)
df_expanded = pd.concat([final_df_embed.drop(columns='embedding'), embedding_df], axis=1)

# Final check
print(df_expanded.shape)
df_expanded.head()

(2521, 537)


,bmi,is_training,jaw_width,face_height,cheekbone_width,nose_width,mouth_width,mouth_height,eye_distance,left_eye_width,...,embedding_502,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511
0,34.207396,1,242.008264,176.045449,239.002092,50.000000,97.020616,18.027756,62.008064,44.045431,...,0.009220,0.010975,-0.032105,0.066220,0.093333,0.015575,-0.044366,-0.028363,0.032760,-0.091712
1,26.453720,1,227.140925,185.067555,220.056811,48.041649,88.051122,22.022716,59.008474,41.012193,...,0.022991,0.011585,-0.036389,0.112116,0.065485,-0.011134,0.051473,-0.040599,-0.018287,-0.001118
2,34.967561,1,269.185809,217.057596,266.030073,48.000000,103.019416,51.000000,59.033889,42.011903,...,0.008165,0.013569,-0.036874,0.096528,0.097352,0.047539,-0.021999,-0.018553,0.033972,-0.122086
3,22.044766,1,244.018442,216.148097,239.002092,44.045431,102.044108,50.039984,59.008474,41.000000,...,-0.013346,0.043490,-0.063243,0.048506,0.087151,0.019768,-0.021870,0.032662,0.014326,-0.072033
4,25.845588,1,238.838858,196.063765,236.541751,48.041649,93.085982,24.020824,69.260378,48.259714,...,0.013444,0.008492,-0.082456,0.055760,0.108606,-0.004980,0.021530,0.034712,0.075459,-0.061361


In [ ]:
X_train = df_expanded[df_expanded["is_training"] == 1].drop(columns=["bmi", "is_training"])
y_train = df_expanded[df_expanded["is_training"] == 1]["bmi"]
X_test = df_expanded[df_expanded["is_training"] == 0].drop(columns=["bmi", "is_training"])
y_test = df_expanded[df_expanded["is_training"] == 0]["bmi"]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2521, 535) (2521,) (0, 535) (0,)


In [ ]:
from sklearn.model_selection import train_test_split

# Drop `is_training` column
df_clean = df_expanded.drop(columns=["is_training"])

# Features and target
X = df_clean.drop(columns=["bmi"])
y = df_clean["bmi"]

# Split into train/test sets (e.g., 80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Confirm shapes
print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (2016, 535) (2016,)
Test: (505, 535) (505,)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

def train_bmi_regressor_with_cv(X, y, cv_folds=5, n_iter=20, random_state=42):
    """
    Trains an XGBoost regressor using RandomizedSearchCV.
    """
    xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=random_state
)

    param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_lambda': [0.1, 1, 10],
        'reg_alpha': [0, 0.1, 1]
    }
    xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=random_state
    )

    scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    search = RandomizedSearchCV(
        xgb,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring=scorer,
        cv=cv_folds,
        verbose=3,
        random_state=random_state,
        n_jobs=-1
    )

    search.fit(X, y)

    print(f"Best MAE: {-search.best_score_:.4f}")
    print(f"Best Parameters: {search.best_params_}")

    return search.best_estimator_

best_bmi_model = train_bmi_regressor_with_cv(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best MAE: 5.4803
Best Parameters: {'subsample': 0.8, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 1.0}


In [ ]:
# Predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr

y_pred = best_bmi_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
#rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
pearson_corr, _ = pearsonr(y_test, y_pred)

print(f"MAE: {mae:.2f}")
#print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")
print(f"Test Pearson Correlation: {pearson_corr:.2f}")

MAE: 5.72
R² Score: 0.15
Test Pearson Correlation: 0.39


Does not perform as well as our XGBoost or MLP Model.